In [1]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.serialization import load_pem_public_key
from cryptography.hazmat.primitives.asymmetric.padding import OAEP, MGF1
from cryptography.hazmat.primitives import serialization
from cryptography.fernet import Fernet

def generate_rsa_keys():
    # Generate private key
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )

    # Generate public key
    public_key = private_key.public_key()
    return private_key, public_key

def encrypt_message_with_public_key(message, public_key):
    encrypted_message = public_key.encrypt(
        message,
        OAEP(
            mgf=MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return encrypted_message

def decrypt_message_with_private_key(encrypted_message, private_key):
    decrypted_message = private_key.decrypt(
        encrypted_message,
        OAEP(
            mgf=MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return decrypted_message

def symmetric_encrypt_message(message, key):
    f = Fernet(key)
    return f.encrypt(message)

def symmetric_decrypt_message(encrypted_message, key):
    f = Fernet(key)
    return f.decrypt(encrypted_message)

# Server Side
server_private_key, server_public_key = generate_rsa_keys()

# Simulate client side
# Client requests server's public key (simulated here by directly accessing it)
# Generate symmetric key for encryption
symmetric_key = Fernet.generate_key()

# Client encrypts the symmetric key using server's public key
encrypted_symmetric_key = encrypt_message_with_public_key(symmetric_key, server_public_key)

# Server decrypts the received symmetric key using its private key
decrypted_symmetric_key = decrypt_message_with_private_key(encrypted_symmetric_key, server_private_key)

# Test the encryption and decryption process with symmetric key
test_message = b"Hello, this is a secret message!"
encrypted_message = symmetric_encrypt_message(test_message, decrypted_symmetric_key)
decrypted_message = symmetric_decrypt_message(encrypted_message, decrypted_symmetric_key)

test_message, encrypted_message, decrypted_message


(b'Hello, this is a secret message!',
 b'gAAAAABlkq12iFgNb7RwPF-b3Wq402_nqaC2gn1pTrjhxAw6Na26S9tL5mkZeSec-6lKLSyMLyVpJpfqsZz7Tu-mtXXItx_4ElwxyM8oZCH14Dov5HwV09QErs557eYwmWcTig2yG_JU',
 b'Hello, this is a secret message!')

In [2]:
import json
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes
from cryptography.fernet import Fernet

# Server Side Code
class Server:
    def __init__(self):
        self.private_key, self.public_key = self.generate_rsa_keys()
        self.symmetric_key = None

    def generate_rsa_keys(self):
        private_key = rsa.generate_private_key(
            public_exponent=65537,
            key_size=2048,
            backend=default_backend()
        )
        public_key = private_key.public_key()
        return private_key, public_key

    def get_public_key(self):
        public_pem = self.public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        )
        return public_pem

    def decrypt_symmetric_key(self, encrypted_key):
        decrypted_key = self.private_key.decrypt(
            encrypted_key,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        self.symmetric_key = decrypted_key

    def decrypt_message(self, encrypted_message):
        if self.symmetric_key:
            f = Fernet(self.symmetric_key)
            return f.decrypt(encrypted_message)
        else:
            return None


# Client Side Code
class Client:
    @staticmethod
    def encrypt_message_with_public_key(message, public_key_pem):
        public_key = load_pem_public_key(public_key_pem, backend=default_backend())
        encrypted_message = public_key.encrypt(
            message,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        return encrypted_message

    @staticmethod
    def symmetric_encrypt_message(message, key):
        f = Fernet(key)
        return f.encrypt(message)


# Testing the Encryption Process
# Simulate Server
server = Server()

# Simulate Client
client = Client()

# Client requests public key from the Server
server_public_key = server.get_public_key()

# Client generates symmetric key
symmetric_key = Fernet.generate_key()

# Client encrypts symmetric key using Server's public key
encrypted_symmetric_key = client.encrypt_message_with_public_key(symmetric_key, server_public_key)

# Client sends encrypted symmetric key to Server
server.decrypt_symmetric_key(encrypted_symmetric_key)

# Client encrypts a message using symmetric key
test_message = b"This is a secret message."
encrypted_message = client.symmetric_encrypt_message(test_message, symmetric_key)

# Server decrypts the message
decrypted_message = server.decrypt_message(encrypted_message)

# Results
test_message, encrypted_message, decrypted_message

(b'This is a secret message.',
 b'gAAAAABlkq12YDv2YRRvPch8bG58MehCrMNimKVY8BYYAZCDf6jez9PZ04G3TW7Ys6G3RUIiFdTIlXr1K8JJ10fcUgPh7C9b3tkS4z5PEEb-BYlInzgvbkc=',
 b'This is a secret message.')